<a href="https://colab.research.google.com/github/thetsuthetsu/PythonSamples/blob/main/pytorch%E5%AD%A6%E7%BF%92%E3%83%AB%E3%83%BC%E3%83%97.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [26]:
from sklearn.datasets import load_iris

In [27]:
iris = load_iris()

In [28]:
# 入力値と目標値を抽出
x = iris['data']
t = iris['target']

In [29]:
# PytorchのTensor型へ変換
x = torch.tensor(x, dtype=torch.float32)
t = torch.tensor(t, dtype=torch.int64)

In [30]:
## DataLoaderの定義：Miniバッチ処理に必要
# 入力値と目標値をまとめる
dataset = torch.utils.data.TensorDataset(x,t)

In [65]:
# データセット分割
# 学習データ(60)、検証データ(20)、テストデータ(20) ^
# 各データのサンプル数を決定
# train : val : test = 60 : 20 : 20
n_train = int(len(dataset) * 0.6)
n_val = int(len(dataset) * 0.2)
n_test = len(dataset) - n_train - n_val

In [78]:
# データセットの分割
torch.manual_seed(0)
train, val, test = torch.utils.data.random_split(dataset, [n_train, n_val, n_test])

In [79]:
# バッチサイズ (目安：学習サンプル数の1/10～1/100)
batch_size = 10

In [80]:
train_loader = torch.utils.data.DataLoader(train, batch_size, shuffle=True, drop_last=True)
val_loader = torch.utils.data.DataLoader(val, batch_size)
test_loader = torch.utils.data.DataLoader(test, batch_size)

x,t = next(iter(train_loader))
x

tensor([[7.7000, 3.8000, 6.7000, 2.2000],
        [5.0000, 3.4000, 1.6000, 0.4000],
        [5.5000, 3.5000, 1.3000, 0.2000],
        [6.4000, 2.8000, 5.6000, 2.2000],
        [6.7000, 2.5000, 5.8000, 1.8000],
        [7.7000, 3.0000, 6.1000, 2.3000],
        [5.1000, 2.5000, 3.0000, 1.1000],
        [5.0000, 3.6000, 1.4000, 0.2000],
        [7.3000, 2.9000, 6.3000, 1.8000],
        [6.1000, 2.8000, 4.0000, 1.3000]])

In [69]:
## ネットワークの定義 (4 -> 4 -> 3の全結合層)
# fc1: input:4 => output:4
# fc2: input:4 => output:3
class Net(nn.Module):
  # 使用するオブジェクトを定義
  def __init__(self):
    super().__init__()
    self.fc1 = nn.Linear(4,4)
    self.fc2 = nn.Linear(4,3)

  # 順伝搬
  def forward(self, x):
    h = self.fc1(x)
    h = F.relu(h)
    h = self.fc2(h)
    return h


In [81]:
# GPUの有無を確認
torch.cuda.is_available()
# デバイスの選択
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

## 学習ループ
# エポック数
max_epoch = 1
torch.manual_seed(0)
net = Net().to(device)

In [84]:
# 最適化手法の定義 (SGD, 学習係数:0.01)
optimizer = torch.optim.SGD(net.parameters(), lr=0.1)

In [85]:
for eposh in range(max_epoch):
  for batch in train_loader:
    x,t = batch
    x = x.to(device)
    t = t.to(device)
    y = net(x)
    loss = F.cross_entropy(y, t)

    # 正解率追加
    y_label = torch.argmax(y, dim=1)
    accuracy = (y_label == t).sum().float() / len(t)
    print(f'accuracy: {accuracy:.2f}')

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

accuracy: 0.80
accuracy: 0.60
accuracy: 0.80
accuracy: 0.50
accuracy: 0.60
accuracy: 0.60
accuracy: 0.40
accuracy: 0.80
accuracy: 0.10


In [86]:
# 正解率を計算する関数
def calc_accuracy(data_loader):
    with torch.no_grad():
      total = 0
      correct = 0.0

      for batch in data_loader:
        x,t =batch
        x = x.to(device)
        t = t.to(device)
        y = net(x)

        y_label = torch.argmax(y, dim=1)
        total += len(t)
        correct += (y_label == t).sum()

      # 全体の平均を算出
      accuracy = correct / total
    return accuracy

In [74]:
calc_accuracy(val_loader)

tensor(0.7000, device='cuda:0')

In [87]:
calc_accuracy(test_loader)

tensor(0.7000, device='cuda:0')